Credit:
This work is heavily borrowed from the following source:
Author: BARAN BINGÖL
Kaggle URL: https://www.kaggle.com/code/baranbingl/face-shape-detection-85-acc-on-test-set

In [ ]:
!pip3 install numpy matplotlib pillow torch torchvision scikit-learn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import random
import os

import torch
import torch.nn as nn
import torchvision
from torchvision import datasets
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torchvision.transforms as T
from torch.cuda.amp import GradScaler
from sklearn.metrics import accuracy_score

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"On device : {device}")

In [ ]:
datasets_path = './datasets'

In [ ]:
oval_training_path = os.path.join(datasets_path, 'training_set/Oval')
example_image_name = random.choice(os.listdir(oval_training_path))
example_image_path = os.path.join(oval_training_path, example_image_name)
example_image = Image.open(example_image_path)

plt.imshow(example_image)
plt.title(f"Image Size: {example_image.size}")
plt.axis('off')
plt.show()

In [ ]:
train_transforms = T.Compose([
    T.Resize((224, 224)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(degrees=10),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transforms = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
BATCH_SIZE = 64
NUM_WORKERS = os.cpu_count() - 1
TRAIN_PATH = os.path.join(datasets_path, './training_set')
TEST_PATH = os.path.join(datasets_path, './testing_set')

def safe_pil_loader(path):
    try:
        with open(path, 'rb') as f:
            img = Image.open(f)
            return img.convert('RGB')
    except (OSError, SyntaxError):
        return Image.new('RGB', (224, 224))

train_dataset = datasets.ImageFolder(root=TRAIN_PATH, transform=train_transforms, loader=safe_pil_loader)
test_dataset = datasets.ImageFolder(root=TEST_PATH, transform=test_transforms, loader=safe_pil_loader)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

In [ ]:
train_dataset.class_to_idx

In [ ]:
dataiter = iter(train_loader)
images, labels = next(dataiter)

example_image = images[0]
example_image = example_image.permute(1, 2, 0).numpy()
example_label = labels[0].item()

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
example_image = example_image * std + mean

plt.imshow(example_image)
plt.title(f"Label: {example_label}")
plt.axis('off')
plt.show()

In [ ]:
dataiter = iter(test_loader)
images, labels = next(dataiter)

example_image = images[0]
example_image = example_image.permute(1, 2, 0).numpy()
example_label = labels[0].item()

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
example_image = example_image * std + mean

plt.imshow(example_image)
plt.title(f"Label: {example_label}")
plt.axis('off')
plt.show()

In [ ]:
torch.manual_seed(42)

model = torchvision.models.efficientnet_b4(pretrained=True)
num_classes = len(train_dataset.classes)
model.classifier = nn.Sequential(
    nn.Dropout(p=0.3, inplace=True),
    nn.Linear(model.classifier[1].in_features, num_classes)
)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-2, betas=(0.9, 0.999), weight_decay=0.01)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

model.to(device)
num_epochs = 25
best_val_loss = 1_000_000

In [ ]:
scaler = GradScaler()

for epoch in range(num_epochs):
    model.train()
    train_losses =  []

    for inputs, labels in train_loader:

        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        with torch.autocast(device_type=device, dtype=torch.float16):
          outputs = model(inputs)
          loss = criterion(outputs, labels)

        scaler.scale(loss).backward()

        scaler.step(optimizer)
        scaler.update()

        train_losses.append(loss.item())

    avg_train_loss = sum(train_losses) / len(train_losses)

    model.eval()
    validation_losses = []
    all_predictions = []
    all_labels = []

    with torch.inference_mode():
      for inputs, labels in test_loader:

        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        validation_losses.append(loss)

        _, predicted = torch.max(outputs.data, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

      avg_val_loss = sum(validation_losses) / len(validation_losses)
      scheduler.step(avg_val_loss)

      accuracy = accuracy_score(all_labels, all_predictions)
      print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.4f}")

      if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        os.makedirs('models', exist_ok=True)
        torch.save(model.state_dict(), 'models/best_model.pth')
print(f'Best Validation Loss: {best_val_loss:.4f}')

In [ ]:
!pip3 uninstall -y numpy matplotlib pillow torch torchvision scikit-learn